In [ ]:
import pickle
import os
import numpy as np
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

CIFAR_DIR = './cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

def loadData(filename):
    with open(filename, "rb") as f:
        data = pickle.load(f, encoding='latin1')
        return data['data'], data['labels']

In [ ]:
class CifarData:
    def __init__(self, filenames, isShuffle):
        allData = []
        allLabels = []
        for filename in filenames:
            data, labels = loadData(filename)
            allData.append(data)
            allLabels.append(labels)
        self._data = np.vstack(allData)
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(allLabels)
        self._size = self._data.shape[0]
        self._startPos = 0
        self._isShuffle = isShuffle
        if isShuffle:
            self.shuffle()
            
    def shuffle(self):
        p = np.random.permutation(self._size)
        self._data = self._data[p]
        self._labels = self._labels[p]

    def nextBatch(self, size):
        endPos = self._startPos + size
        if endPos > self._size:
            if self._isShuffle:
                self.shuffle()
                self._startPos = 0
                endPos = size
            else:
                raise Exception("No more samples")
            
        if endPos > self._size:
            raise Exception("Sample size is less than %d", size)
        
        batch_data = self._data[self._startPos: endPos]
        batch_labels = self._labels[self._startPos: endPos]
        self._startPos = endPos
        return batch_data, batch_labels

train_filenames = [os.path.join(CIFAR_DIR, "data_batch_%d" % i) for i in range(1, 6)]
train_data = CifarData(train_filenames, True)
# print(training_data._size)
# print(training_data.nextBatch(10000))

test_filenames = [os.path.join(CIFAR_DIR, "test_batch")]
test_data = CifarData(test_filenames, False)
# print(test_data._size)
# print(test_data.nextBatch(2000))

In [ ]:
def inception_block(x, output_channels, name):
    with tf.variable_scope(name):
        conv1_1 = tf.layers.conv2d(x, output_channels[0], (1, 1), padding = "same", activation = tf.nn.relu, name = "conv1_1")
        conv3_3 = tf.layers.conv2d(x, output_channels[1], (3, 3), padding = "same", activation = tf.nn.relu, name = "conv3_3")
        conv5_5 = tf.layers.conv2d(x, output_channels[2], (5, 5), padding = "same", activation = tf.nn.relu, name = "conv5_5")
        pooling = tf.layers.max_pooling2d(x, (2, 2), (2, 2), name = "pool")
        input_shape = x.get_shape().as_list()[1:]
        pooling_shape = pooling.get_shape().as_list()[1:]
        padding_width = (input_shape[0] - pooling_shape[0]) // 2
        padding_height = (input_shape[1] - pooling_shape[1]) // 2
        pooling_pad = tf.pad(pooling, [[0, 0], [padding_width, padding_width], [padding_height, padding_height], [0, 0]])
        return tf.concat([conv1_1, conv3_3, conv5_5, pooling_pad], axis = 3)

def incpetion_net(x):
    conv1 = tf.layers.conv2d(x, 32, (3, 3), padding = "same", activation = tf.nn.relu, name = "conv1")
    pooling1 = tf.layers.max_pooling2d(conv1, (2, 2), (2, 2), name = "pool1")
    inception_2a = inception_block(pooling1, [16, 16, 16], "inception_2a")
    inception_2b = inception_block(inception_2a, [16, 16, 16], "inception_2b")
    pooling2 = tf.layers.max_pooling2d(inception_2b, (2, 2), (2, 2), name = "pool2")
    inception_3a = inception_block(pooling2, [16, 16, 16], "inception_3a")
    inception_3b = inception_block(inception_3a, [16, 16, 16], "inception_3b")
    pooling3 = tf.layers.max_pooling2d(inception_3b, (2, 2), (2, 2), name = "pool3")
    logits = tf.layers.flatten(pooling3)
    return logits

x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])

x_reshape = tf.reshape(x, [-1, 3, 32, 32])
x_reshape = tf.transpose(x_reshape, perm = [0, 2, 3, 1])

inception = incpetion_net(x_reshape)
y_ = tf.layers.dense(inception, 10)

loss = tf.losses.sparse_softmax_cross_entropy(labels = y, logits = y_)

predict = tf.argmax(y_, 1)
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with (tf.name_scope("train_op")):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)


In [ ]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.nextBatch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict = {
                x: batch_data,
                y: batch_labels
            }
        )
        
        if (i + 1) % 500 == 0:
            print("[Train] Step: %d, loss: %4.5f, acc: %4.5f" % (i + 1, loss_val, acc_val))
            
        if (i + 1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.nextBatch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels
                    }
                )
                all_test_acc_val.append(test_acc_val)
            test_acc_val = np.mean(all_test_acc_val)
            print("[Test ] Step: %d, acc: %4.5f" % (i + 1, test_acc_val))